In [58]:
import numpy as np
import pandas as pd
import  math
from scipy import optimize
from itertools import combinations

In [59]:
from sklearn import linear_model
from sklearn import ensemble
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score,mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier


In [60]:
groups = pd.read_csv('groupstage.csv')
data = pd.read_csv('Past_International_Matches_Points.csv')


In [61]:
data['result'] = (data['home_team_result'] == 'Win') * 1 
data['neutral'] = (data['neutral_location'] == 'TRUE') * 1

In [62]:
X = data.loc[:, ['mean_coeff', 'home_team_fifa_rank', 'away_team_fifa_rank', 'importance','neutral']]
y = data['result']
X

,mean_coeff,home_team_fifa_rank,away_team_fifa_rank,importance,neutral
0,1.000,59,22,2.5,0
1,0.925,8,14,1.0,0
2,1.000,35,94,2.5,0
3,0.850,65,86,1.0,0
4,1.000,67,5,2.5,0
...,...,...,...,...,...
23916,0.990,180,153,2.5,0
23917,0.990,192,135,2.5,0
23918,0.925,28,60,1.0,0
23919,0.850,23,35,1.0,0


In [63]:
m, n = X.shape

X = np.concatenate([np.ones((m, 1)), X], axis=1)
X

array([[ 1.   ,  1.   , 59.   , 22.   ,  2.5  ,  0.   ],
       [ 1.   ,  0.925,  8.   , 14.   ,  1.   ,  0.   ],
       [ 1.   ,  1.   , 35.   , 94.   ,  2.5  ,  0.   ],
       ...,
       [ 1.   ,  0.925, 28.   , 60.   ,  1.   ,  0.   ],
       [ 1.   ,  0.85 , 23.   , 35.   ,  1.   ,  0.   ],
       [ 1.   ,  0.85 , 29.   , 32.   ,  1.   ,  0.   ]])

In [64]:
def sigmoid(z):
    z = np.array(z)
    g = np.zeros(z.shape)
    g = 1 / (1 + np.exp(-z))
    return g

In [65]:
# def costFunction(theta, X, Y):

#      m = Y.size 

#      J = 0
#      grad = np.zeros(theta.shape)

#      h = sigmoid(X.dot(theta.T))

#      J = (-1/m) * sum((Y*np.log(h)) + (1-Y)*np.log(1-h))
#      grad = (1/m) * (h - Y).dot(X)
#      return J, grad

In [66]:
# initial_theta = np.zeros(n+1)
# options= {'maxfun': 400}


# res = optimize.minimize(costFunction,
#                          initial_theta,
#                          (X, Y),
#                          jac=True,
#                          method='TNC',
#                          options=options)


# cost = res.fun

# theta = res.x

In [67]:
def predict(theta, X):
    m = (X).shape[0] 
    p = np.zeros(m)

    h = sigmoid((X).dot(theta.T))

    p = np.round(h)
    
    return p

In [68]:
# prob = sigmoid(np.dot(X[0], theta))
# print(prob)
# p = predict(theta, X)
# print(np.mean(p == Y) * 100)

In [69]:
def costFunctionReg(theta, X, Y, lambda_):
   
    m = Y.size  

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid((X).dot(theta.T))
    theta_ = theta
    theta_[0] = 0
    J = -(1/m)*(np.sum(Y*np.log(h) + (1-Y)*np.log(1-h))) + (lambda_/(2*m)) * (np.sum(np.square(theta_)))
    grad = (1/m) * (h - Y).dot(X)
    
    grad = (1/m)* (h-Y).dot(X) + (lambda_ / m)*(theta_)
    
    return J, grad

In [70]:
def learn(X, Y):
    initial_theta = np.random.rand(6) * 2 * 0.12 - 0.12

    lambda_ = 1

    options= {'maxfun': 100}

    res = optimize.minimize(costFunctionReg,
                            initial_theta,
                            (X, Y, lambda_),
                            jac=True,
                            method='TNC',
                            options=options 
                            )

    cost = res.fun
    theta = res.x

    p = predict(theta, X)

    return p,theta

In [81]:
learn(X,y)
prob = sigmoid(np.dot((X[0]), theta))
print(prob)
p = predict(theta, X)
p
print(np.mean(p == y) * 100)
auc_score = metrics.roc_auc_score(y,predict(theta, X))
print(auc_score)
mean_error = mean_absolute_error(y, predict(theta, X))
print(mean_error)

0.287821977487956
67.61004974708416
0.6757704223705467
0.3238995025291585


## Group Stage set-up

In [72]:
def splitInGroups(groups_use):
    groups_sorted = groups_use.sort_values(by =['Group', 'points'], ascending = [True, False]) 
    print(groups_sorted)

    A = groups_sorted[groups_sorted['Group'] == 'A']
    B = groups_sorted[groups_sorted['Group'] == 'B']
    C = groups_sorted[groups_sorted['Group'] == 'C']
    D = groups_sorted[groups_sorted['Group'] == 'D']
    E = groups_sorted[groups_sorted['Group'] == 'E']
    F = groups_sorted[groups_sorted['Group'] == 'F']
    G = groups_sorted[groups_sorted['Group'] == 'G']
    H = groups_sorted[groups_sorted['Group'] == 'H']

    return A, B, C, D, E, F, G, H

In [73]:
group_use = groups.set_index(['Country_Name'])
def group_stage(mlp, groups, theta):
    group_use['points'] = 0
    for group in set(groups['Group']):
        print('====Group {}===='.format(group))
        for home, away in combinations (groups.query('Group == "{}"'.format(group)).values, 2):
            print("{} vs {} : " .format(home[0], away[0], end=''))
            row = np.zeros(6) #, columns=['mean_coeff', 'home_team_fifa_rank', 'away_team_fifa_rank',
        #'importance', 'neutral']

            home = home[0]
            away = away[0]

            home_coeff = group_use.loc[home, 'conf_coeff']
            away_coeff = group_use.loc[away, 'conf_coeff']
            home_rank = group_use.loc[home, 'Rank']
            away_rank = group_use.loc[away, 'Rank']

            row[0] = 1 #bias
            row[1] = (home_coeff + away_coeff)/2
            row[2] = home_rank
            row[3] = away_rank
            row[4] = 4
            row[5] = 1

            #PUT MODEL OUTPUT AFTER
            prediction_model = predict(theta, row)
            # prob = sigmoid(np.dot(row, theta))[:][0]
            points = 0 
            if prediction_model == 1 :
                group_use.loc[home, 'points'] += 3
                print("{} wins".format(home))
                
            elif prediction_model == 0:
                group_use.loc[away, 'points'] += 3
                print("{} wins".format(away))
                
            # elif prediction_model == 3 :
            #     group_use.loc[home, 'points'] += 3
            #     print("{} wins".format(home))
    
    return splitInGroups(group_use)
        
        

## Knockout Stages

In [74]:
def match_winner(mlp, away, home, theta): # World Cup importance: 4, always neutral
    row = np.zeros(6)
    row[0] = 1 #bias
    row[1] = (home['conf_coeff'].values[0] + away['conf_coeff'].values[0]) / 2
    row[2] = home['Rank'].values[0]
    row[3] = away['Rank'].values[0]
    row[4] = 4 #importance
    row[5] = 1 #neutral

    prediction_model = predict(theta, row)

    if prediction_model == 1:
        print(home.index.values + " vs " + away.index.values + ". Winner " + home.index.values)
        return home, away   # winner, loser
    else:
        print(home.index.values + " vs " + away.index.values + ". Winner " + away.index.values)
        return away, home

## LAST 16


In [75]:
def last16(mlp, A, B, C, D, E, F, G, H, theta):
    # get the top two of each group
    a1 = A.iloc[0:1]
    a2 = A.iloc[1:2]

    b1 = B.iloc[0:1]
    b2 = B.iloc[1:2]

    c1 = C.iloc[0:1]
    c2 = C.iloc[1:2]

    d1 = D.iloc[0:1]
    d2 = D.iloc[1:2]

    e1 = E.iloc[0:1]
    e2 = E.iloc[1:2]

    f1 = F.iloc[0:1]
    f2 = F.iloc[1:2]

    g1 = G.iloc[0:1]
    g2 = G.iloc[1:2]

    h1 = H.iloc[0:1]
    h2 = H.iloc[1:2]

    # simulate winners
    l16_1,_ = match_winner(mlp, a1,b2, theta)
    l16_2,_ = match_winner(mlp, c1,d2, theta)
    l16_3,_ = match_winner(mlp, e1,f2, theta)
    l16_4,_ = match_winner(mlp, g1,h2, theta)
    l16_5,_ = match_winner(mlp, b1,a2, theta)
    l16_6,_ = match_winner(mlp, d1,c2, theta)
    l16_7,_ = match_winner(mlp, f1,e2, theta)
    l16_8,_ = match_winner(mlp, h1,g2, theta)

    return l16_1, l16_2, l16_3, l16_4, l16_5, l16_6, l16_7, l16_8

## Quarter Finals

In [76]:
def quaterFinals(mlp, l16_1, l16_2, l16_3, l16_4, l16_5, l16_6, l16_7, l16_8, theta):
    qf_1,_ = match_winner(mlp, l16_1, l16_2, theta)
    qf_2,_ = match_winner(mlp, l16_3, l16_4, theta)
    qf_3,_ = match_winner(mlp, l16_5, l16_6, theta)
    qf_4,_ = match_winner(mlp, l16_7, l16_8, theta)

    return qf_1, qf_2, qf_3, qf_4

## Semi Finals

In [77]:
def semiFinals(mlp, qf_1, qf_2, qf_3, qf_4, theta):
    sf_1, l_1 = match_winner(mlp, qf_1, qf_2, theta)
    sf_2, l_2 = match_winner(mlp, qf_3, qf_4, theta)

    return sf_1, sf_2, l_1, l_2

## Final

In [78]:
def finals(mlp, sf_1, sf_2, l_1, l_2, theta):
    third_place, fourth_place = match_winner(mlp, l_1, l_2, theta)
    first_place, second_place = match_winner(mlp, sf_1, sf_2, theta)

    return first_place, second_place, third_place, fourth_place

## SIMULATOR

In [79]:
group_use['Wins'] = 0
group_use['Runners up'] = 0
group_use['Third place'] = 0
group_use['Fourth place'] = 0

for i in range(1):
    print("============================iter" + str(i) + "============================")
    mlp, theta = learn(X, y)
    A, B, C, D, E, F, G, H = group_stage(mlp, groups, theta)
    l16_1, l16_2, l16_3, l16_4, l16_5, l16_6, l16_7, l16_8 = last16(mlp, A, B, C, D, E, F, G, H, theta)
    qf_1, qf_2, qf_3, qf_4 = quaterFinals(mlp, l16_1, l16_2, l16_3, l16_4, l16_5, l16_6, l16_7, l16_8, theta)
    sf_1, sf_2, l_1, l_2 = semiFinals(mlp, qf_1, qf_2, qf_3, qf_4, theta)
    first_place, second_place, third_place, fourth_place = finals(mlp, sf_1, sf_2, l_1, l_2, theta)

    group_use.loc[first_place.index.values, 'Wins'] += 1
    group_use.loc[second_place.index.values, 'Runners up'] += 1
    group_use.loc[third_place.index.values, 'Third place'] += 1
    group_use.loc[fourth_place.index.values, 'Fourth place'] += 1

============================iter0============================
====Group A====
Qatar vs Ecuador : 
Ecuador wins
Qatar vs Senegal : 
Senegal wins
Qatar vs Netherlands : 
Netherlands wins
Ecuador vs Senegal : 
Senegal wins
Ecuador vs Netherlands : 
Netherlands wins
Senegal vs Netherlands : 
Netherlands wins
====Group D====
France vs Denmark : 
France wins
France vs Tunisia : 
France wins
France vs Australia : 
France wins
Denmark vs Tunisia : 
Denmark wins
Denmark vs Australia : 
Denmark wins
Tunisia vs Australia : 
Tunisia wins
====Group G====
Brazil vs Serbia : 
Brazil wins
Brazil vs Switzerland : 
Brazil wins
Brazil vs Cameroon : 
Brazil wins
Serbia vs Switzerland : 
Switzerland wins
Serbia vs Cameroon : 
Serbia wins
Switzerland vs Cameroon : 
Switzerland wins
====Group H====
Portugal vs Ghana : 
Portugal wins
Portugal vs Uruguay : 
Portugal wins
Portugal vs Korea Republic : 
Portugal wins
Ghana vs Uruguay : 
Uruguay wins
Ghana vs Korea Republic : 
Korea Republic wins
Uruguay vs Korea 

In [80]:
group_use[['Wins', 'Runners up', 'Third place', 'Fourth place']]

,Wins,Runners up,Third place,Fourth place
Country_Name,,,,
Qatar,0,0,0,0
Ecuador,0,0,0,0
Senegal,0,0,0,0
Netherlands,0,0,0,1
England,0,1,0,0
IR Iran,0,0,0,0
USA,0,0,0,0
Wales,0,0,0,0
Argentina,0,0,0,0
